In [1]:
import sys

sys.path.append('../')

from model_v3 import  *
from data import *
import os
import cv2
import keras
import numpy as np
from keras.callbacks import TensorBoard
import tensorflow as tf
import keras.backend.tensorflow_backend as K
import matplotlib.pyplot as plt

from utils import helpers
from utils import utils

Using TensorFlow backend.


In [2]:
train_path = "../640x400_small"
image_folder = "train"
label_folder = "trainannot"
valid_path =  "../640x400_small"
valid_image_folder ="val"
valid_label_folder = "valannot"
test_path = '../640x400_small/test/'
test_label_folder = '../640x400_small/testannot/'
log_filepath = '../log'
weights_path = '../model/model_v3_redux_8.hdf5'
flag_multi_class = True
num_classes = 4

dp = data_preprocess(
    train_path=train_path,image_folder=image_folder,label_folder=label_folder,
    valid_path=valid_path,valid_image_folder=valid_image_folder,valid_label_folder=valid_label_folder,
    flag_multi_class=flag_multi_class, num_classes=num_classes, test_label_folder=test_label_folder,
    test_path=test_path, img_rows=640, img_cols=400, csv_name='class_dict.csv'
)

In [3]:
model = unet(pretrained_weights=weights_path, num_class=4, input_size=(640, 400, 3))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 640, 400, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 640, 400, 8)  224         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 640, 400, 8)  584         conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxP

In [4]:
test_data = dp.testGenerator()
test_label = dp.labelTestGenerator()

mean_iou = []

In [5]:
for image, label in zip(test_data, test_label):
    pred = model.predict(image)
    pred = helpers.reverse_one_hot(pred[0])
    pred = helpers.colour_code_segmentation(pred, dp.label_values)
    
    iou = utils.compute_mean_iou(pred, label[0])
    mean_iou.append(iou)

print('mean iou:', np.mean(mean_iou))

mean iou: 0.046233601596665126


In [6]:
print('max iou:', np.max(mean_iou))

max iou: 0.07011879006410257


In [7]:
print('min iou:', np.min(mean_iou))

min iou: 0.04128001302083333
